# Autoregressive RNN with continuous input
- This model is autoregressive, which means that the prediction of the last time step is the input of the current time step. This way the model can predict a varying number of steps into the future without retraining
- Some features, like time and weather, are fed into the model from outside even during prediction phase so the model does not have to predict those by itself

Based on: https://www.tensorflow.org/tutorials/structured_data/time_series

In [ ]:
import os
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import IPython
import IPython.display

sns.set_theme()

### Import data

In [ ]:
# df = pd.read_pickle("..\data\data_prepared.pkl")
df = pd.read_pickle("../data/prepared/electricity+time_2017-2021.pkl")
df.head()

### Split the data
- 70% training
- 20% validation
- 10% test

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
df_train = df[0:int(n*0.7)]
df_val = df[int(n*0.7):int(n*0.9)]
df_test = df[int(n*0.9):]

### Normalize the data

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler

scaler = MinMaxScaler()
# fit scaler to training data
scaler.fit(df_train)
# scale all sets according to train set, preserve data frames
df_train = pd.DataFrame(scaler.transform(df_train),
                        columns=df.columns, index=df_train.index)
df_val = pd.DataFrame(scaler.transform(df_val),
                        columns=df.columns, index=df_val.index)
df_test = pd.DataFrame(scaler.transform(df_test),
                        columns=df.columns, index=df_test.index)

In [ ]:
len(df_train.columns)

### Data Windowing
#### 1. Indexes and offsets

- Windows Generator that can create multiple inputs, one for the inputs that are only known in the past (power production) and one for inputs that are also known in the future (time and weather)

In [ ]:
class WindowGenerator():
    def __init__(self, past_width, future_width,
                 label_columns, precise_columns=None, forecast_columns=None,
                 train_df=df_train, val_df=df_val, test_df=df_test):

        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Check if length of provided features match with data frame
        num_columns = 0
        num_columns += len(label_columns)
        if precise_columns is not None:
            num_columns += len(precise_columns)
        if forecast_columns is not None:
            num_columns += len(forecast_columns)
        assert num_columns == len(train_df.columns), \
            "Length of provided label, precise, and forecast features do not match data frame"

        # Work out the column indices
        self.columns_indices = {name: i for i, name in
                                enumerate(train_df.columns)}
        self.label_columns = label_columns
        self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
        self.precise_columns = precise_columns
        self.forecast_columns = forecast_columns

        # Work out the window parameters.
        self.past_width = past_width
        self.future_width = future_width

        self.total_window_size = past_width + future_width

        self.input_past_slice = slice(0, past_width)
        self.input_past_indices = np.arange(self.total_window_size)[
            self.input_past_slice]

        self.input_future_slice = slice(self.past_width, -1)
        self.input_future_indices = np.arange(self.total_window_size)[
            self.input_future_slice]

        self.label_slice = slice(self.past_width, None)
        self.label_indices = np.arange(self.total_window_size)[
            self.label_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size:    {self.total_window_size}',
            f'Input past indices:   {self.input_past_indices}',
            f'Input future indices: {self.input_future_indices}',
            f'Label indices:        {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
        # f'Precise column name(s):  {self.precise_columns}',
        # f'Forecast column name(s): {self.forecast_columns}'])


#### 2. Split

In [ ]:
def split_window(self, features):
    # Past: All inputs are known
    inputs_past = features[:, self.input_past_slice, :]

    # Future: Only precise and forecast inputs are known
    if self.precise_columns is None:
        self.precise_columns = []
    if self.forecast_columns is None:
        self.forecast_columns = []
    self.input_future_columns = self.precise_columns + self.forecast_columns

    inputs_future = tf.stack(
        [features[:, self.input_future_slice, self.columns_indices[name]]
            for name in self.input_future_columns],
        axis=-1)

    labels = tf.stack(
        [features[:, self.label_slice, self.columns_indices[name]]
            for name in self.label_columns],
        axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs_past.set_shape([None, self.past_width, None])
    inputs_future.set_shape([None, self.future_width-1, None])
    labels.set_shape([None, self.future_width, None])

    # Return inputs and labels
    # The past and future input tuple will be unpacked in the model.call() method
    return (inputs_past, inputs_future), labels


WindowGenerator.split_window = split_window


#### 3. Plot

In [ ]:
def plot(self, model=None, plot_col="prod_solar", max_subplots=3):
    inputs, labels = self.example
    (inputs_past, inputs_future) = inputs
    plt.figure(figsize=(12, 8))
    plot_col_index = self.columns_indices[plot_col]
    max_n = min(max_subplots, len(inputs_past))
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)

        # Plot past inputs
        plt.plot(self.input_past_indices, inputs_past[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=3)

        label_col_index = self.label_columns_indices.get(plot_col, None)

        # Add last point of past array to future arrays to connect lines
        label_indices_plot = np.insert(
            self.label_indices, 0, self.input_past_indices[-1])
        labels_plot = np.insert(
            labels[n, :, label_col_index], 0, inputs_past[n, :, plot_col_index][-1])

        # Plot labels
        plt.plot(label_indices_plot, labels_plot,
                 'C2', label='Labels', marker='.', zorder=1)

        # Plot prediction
        if model is not None:
            predictions_plot = (model(inputs))[n, :, label_col_index]
            predictions_plot = np.insert(
                predictions_plot, 0, inputs_past[n, :, plot_col_index][-1])
            plt.plot(label_indices_plot, predictions_plot,
                     'C1', label='Predictions', marker='.', zorder=2)

        # x Ticks every 6 hours
        plt.xticks(np.arange(self.input_past_indices[0], self.label_indices[-1]+2, 6))

        plt.ylabel(f'{plot_col} [normed]')
        if n == 0:
            plt.legend()

    plt.xlabel('Time [h]')


WindowGenerator.plot = plot


#### 4. Create `tf.data.Datasets`

In [ ]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds


WindowGenerator.make_dataset = make_dataset


In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)


@property
def val(self):
    return self.make_dataset(self.val_df)


@property
def test(self):
    return self.make_dataset(self.test_df)


@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.test` dataset
        result = next(iter(self.test))
        # And cache it for next time
        self._example = result
    return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example


## Window creation

In [ ]:
# Choose which features are only known for the past. These features will be predicted.
prediction_features = ['prod_wind', 'prod_solar']
num_predictions = len(prediction_features)

# Choose which features are precisely known to the model at all time steps
precise_features = ['day_sin', 'day_cos', 'year_sin',
                    'year_cos', 'solar_el', 'solar_el_clip']

# Choose which features are only imprecisely known to the model at future time steps
# NOISE NOT YET IMPLEMENTED
forecast_features = None

PAST_STEPS = 24
PREDICTION_STEPS = 12

multi_window = WindowGenerator(
    past_width=PAST_STEPS, future_width=PREDICTION_STEPS,
    label_columns=prediction_features,
    precise_columns=precise_features,
    forecast_columns=forecast_features
)

multi_window


Now, the `WindowGenerator` object gives access to the tf.data.Dataset objects, to easily iterate over the data.

The `Dataset.element_spec` property tells the structure, data types, and shapes of the dataset elements.

In [ ]:
multi_window.train.element_spec

Iterating over a `Dataset` yields concrete batches:

In [ ]:
for (example_inputs_past, example_inputs_future), example_labels in multi_window.train.take(1):
  print(f'Inputs past shape (batch, time, features): {example_inputs_past.shape}')
  print(f'Inputs fut. shape (batch, time, features): {example_inputs_future.shape}')
  print(f'Labels shape      (batch, time, features): {example_labels.shape}')

Plot an example window:

In [ ]:
multi_window.plot(plot_col="prod_solar")
# multi_window.plot(plot_col="prod_wind")

## Autoregressive LSTM
The model predicts individual time steps that are fed back into itself, so that the model can produce output with a varying length.
This is implemented in the custom model class `FeedBack`.

In [ ]:
class FeedBack(tf.keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.rnn_cell = tf.keras.layers.LSTMCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.rnn_layer = tf.keras.layers.RNN(self.rnn_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_predictions)


In [ ]:
arlstm_model = FeedBack(units=32, out_steps=PREDICTION_STEPS)


#### Model warmup
The warmup method initializes the model's internal state based on the full input from the past (power data, time and weather)

In [ ]:
def warmup(self, inputs_past):
    # inputs.shape => (batch, time, features)
    # x.shape => (batch, lstm_units)
    x, *state = self.rnn_layer(inputs_past)

    # predictions.shape => (batch, features)
    prediction = self.dense(x)
    return prediction, state


FeedBack.warmup = warmup


This method returns a single time-step prediction and the internal state of the `LSTM`:

In [ ]:
prediction, state = arlstm_model.warmup(multi_window.example[0][0])
prediction.shape


#### Model call
A custom call is used to feed the model the full input from the past during warmup and then only the time and weather forecast during the prediction phase, where the model takes the prediction from the last time step to substitute the missing input.

In [ ]:
def call(self, inputs, training=None):
    # unpack the past and future inputs
    inputs_past, inputs_future = inputs
    # Use a TensorArray to capture dynamically unrolled outputs.
    predictions = []
    # Past phase: Initialize the LSTM state
    prediction, state = self.warmup(inputs_past)
    # Insert the first prediction.
    predictions.append(prediction)

    # Future phase: Run the rest of the prediction steps
    for n in range(self.out_steps-1):
        # Select the known input at the current time step
        input = inputs_future[:, n, :]
        # Use the last prediction as unknown input and combine it with the known input
        # x.shape => (batch, features)
        x = tf.concat([prediction, input], axis=1)
        # Execute one lstm step.
        x, state = self.rnn_cell(x, states=state,
                                  training=training)
        # Convert the lstm output to a prediction.
        prediction = self.dense(x)
        # Add the prediction to the output.
        predictions.append(prediction)

    # predictions.shape => (time, batch, features)
    predictions = tf.stack(predictions)
    # predictions.shape => (batch, time, features)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions


FeedBack.call = call


Run on example input:

In [ ]:
print('Output shape (batch, time, features): ',
      arlstm_model(multi_window.example[0]).shape)


In [ ]:
arlstm_model.call(multi_window.example[0])
arlstm_model.summary()


#### Training procedure

In [ ]:
from tensorflow_addons.metrics.r_square import RSquare
import os

def compile_and_fit(model, window, epochs=100, save=False, stop_early=True, model_name=None, patience=10, verbose='auto'):
    callbacks = []
    if save:
        assert model_name is not None, "No model name provided"
        checkpoint_folder = '../models/'+model_name
        if not os.path.exists(checkpoint_folder):
            os.makedirs(checkpoint_folder)
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_folder+'/ckp-weights.hdf5',
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callbacks.append(model_checkpoint)
    if stop_early:
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=patience,
                                                          mode='min',
                                                          verbose=1,
                                                          restore_best_weights=True)
        callbacks.append(early_stopping)

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(
        learning_rate=5e-4), metrics=['mae', RSquare()])

    history = model.fit(window.train, epochs=epochs,
                        validation_data=window.val,
                        callbacks=callbacks,
                        verbose=verbose)
    return history


val_performance = {}
test_performance = {}
history = {}


In [ ]:
model_name = 'ARLSTM 32'
history[model_name] = compile_and_fit(arlstm_model, multi_window, save=True, model_name='ARLSTM32')
IPython.display.clear_output()

val_performance[model_name] = arlstm_model.evaluate(multi_window.val, verbose=0)
test_performance[model_name] = arlstm_model.evaluate(multi_window.test, verbose=1)


In [ ]:
multi_window.plot(arlstm_model, plot_col="prod_wind")

#### Plot Training Losses

In [ ]:
history['ARLSTM 32'].history.keys()

In [ ]:
def model_loss(history):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'], label='Training loss (MSE)')
    plt.plot(history.history['val_loss'], label='Validation loss (MSE)')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.xlabel('epochs')
    plt.legend(loc='upper left')


In [ ]:
model_loss(history['ARLSTM 32'])

## Autoregressive LSTM with Weather Data
- RNN with one LSTM layer with 32 units

Import weather data and add to data frame. Each pickle contains one weather parameter measured at multiple stations.

In [ ]:
# Features that will be extracted from the EM data and their shorter handle
wx_params = {
    "temperature_air_mean_200": "temp",
    "wind_speed": "wind",
    "pressure_air_site": "press",
    "precipitation_height": "precip",
    "sunshine_duration": "sun"
}

# Concatenate all weather parameters into one data frame
wx_list = []
for name, handle in wx_params.items():
    # wx_list.append(pd.read_pickle("../data/weather/cleaned/"+name+".pkl").add_prefix(handle+"_"))
    wx_list.append(pd.read_pickle("../data/weather/long/cleaned/"+name+".pkl").add_prefix(handle+"_"))

wx = pd.concat(wx_list, axis=1, ignore_index=False, verify_integrity=True)
wx.head()


Combine weather data with previous Electricity Map and Time data frame, split and normalize:

In [ ]:
df2 = pd.concat([df, wx], axis=1, ignore_index=False, verify_integrity=True)

column_indices = {name: i for i, name in enumerate(df2.columns)}

n = len(df2)
df2_train = df2[0:int(n*0.7)]
df2_val = df2[int(n*0.7):int(n*0.9)]
df2_test = df2[int(n*0.9):]

scaler2 = RobustScaler()
# fit scaler to training data
scaler2.fit(df2_train)
# scale all sets according to train set, preserve data frames
df2_train = pd.DataFrame(scaler2.transform(df2_train),
                        columns=df2.columns, index=df2_train.index)
df2_val = pd.DataFrame(scaler2.transform(df2_val),
                        columns=df2.columns, index=df2_val.index)
df2_test = pd.DataFrame(scaler2.transform(df2_test),
                        columns=df2.columns, index=df2_test.index)

len(df2_train.columns)

Creating a new window that includes weather data:

In [ ]:
len(wx.columns)

In [ ]:
# Choose which features are only known for the past. These features will be predicted.
# prediction_features same as before

# Choose which features are precisely known to the model at all time steps
# precise_features same as before

# Choose which features are only imprecisely known to the model at future time steps
# NOISE NOT YET IMPLEMENTED
forecast_features = list(wx.columns)

PAST_STEPS = 24
PREDICTION_STEPS = 12

weather_window = WindowGenerator(
    past_width=PAST_STEPS, future_width=PREDICTION_STEPS,
    label_columns=prediction_features,
    precise_columns=precise_features,
    forecast_columns=forecast_features,
    train_df=df2_train, val_df=df2_val, test_df=df2_test
)

weather_window


Inspect the structure of the newly created `weather_window`:

In [ ]:
weather_window.train.element_spec

Build model:

In [ ]:
arlstm_wx_model = FeedBack(units=32, out_steps=PREDICTION_STEPS)

# Run on example input
print('Output shape (batch, time, features): ',
      arlstm_wx_model(weather_window.example[0]).shape)

Train the model:

In [ ]:
model_name = 'ARLSTM 32 + wx'

history[model_name] = compile_and_fit(arlstm_wx_model, weather_window)
IPython.display.clear_output()

val_performance[model_name] = arlstm_wx_model.evaluate(weather_window.val, verbose=0)
test_performance[model_name] = arlstm_wx_model.evaluate(weather_window.test, verbose=1)

model_loss(history[model_name])


In [ ]:
weather_window.plot(arlstm_wx_model, plot_col="prod_wind")

## Autoregressive GRU with Weather Data
- RNN with one GRU layer with 32 units

Modify the `FeedBack` class:

In [ ]:
class FeedBackGRU(FeedBack):
    def __init__(self, units, out_steps):
        super().__init__(units, out_steps)
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = tf.keras.layers.GRUCell(units)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = tf.keras.layers.Dense(num_predictions)


In [ ]:
model_name = 'ARGRU 32 + wx'

argru_wx_model = FeedBackGRU(units=32, out_steps=PREDICTION_STEPS)

history[model_name] = compile_and_fit(argru_wx_model, weather_window)
IPython.display.clear_output()

val_performance[model_name] = argru_wx_model.evaluate(weather_window.val, verbose=0)
test_performance[model_name] = argru_wx_model.evaluate(weather_window.test, verbose=1)

model_loss(history[model_name])

In [ ]:
weather_window.plot(argru_wx_model, plot_col="prod_wind")

## Bigger AR LSTM with Weather Data
Same inputs as previous model, more units (128 instead of 32)

In [ ]:
model_name = 'ARLSTM 128 + wx'

arlstm128_wx_model = FeedBack(units=128, out_steps=PREDICTION_STEPS)

history[model_name] = compile_and_fit(arlstm128_wx_model, weather_window)
IPython.display.clear_output()

val_performance[model_name] = arlstm128_wx_model.evaluate(weather_window.val, verbose=0)
test_performance[model_name] = arlstm128_wx_model.evaluate(weather_window.test, verbose=1)

model_loss(history[model_name])


In [ ]:
# weather_window.plot(arlstm128_wx_model, plot_col="prod_wind")

## Stacked LSTM
- This LSTM consists of two layers witch 96 and 32 units

In [ ]:
class FeedBackStacked(tf.keras.Model):
    def __init__(self, units1, units2, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units1 = units1
        self.units2 = units2
        self.rnn_cell1 = tf.keras.layers.LSTMCell(units1)
        self.rnn_cell2 = tf.keras.layers.LSTMCell(units2)
        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.rnn_layer1 = tf.keras.layers.RNN(
            self.rnn_cell1, return_state=True, return_sequences=True)
        self.rnn_layer2 = tf.keras.layers.RNN(
            self.rnn_cell2, return_state=True)
        self.dense = tf.keras.layers.Dense(num_predictions)

    def warmup(self, inputs_past):
        # inputs.shape => (batch, time, features)
        # x.shape => (batch, lstm_units)
        x, *state1 = self.rnn_layer1(inputs_past)
        x, *state2 = self.rnn_layer2(x)
        # predictions.shape => (batch, features)
        prediction = self.dense(x)
        return prediction, state1, state2

    def call(self, inputs, training=None):
        # unpack the past and future inputs
        inputs_past, inputs_future = inputs
        # Use a TensorArray to capture dynamically unrolled outputs.
        predictions = []
        # Past phase: Initialize the LSTM state
        prediction, state1, state2 = self.warmup(inputs_past)
        # Insert the first prediction.
        predictions.append(prediction)
        # Future phase: Run the rest of the prediction steps
        for n in range(1, self.out_steps):
            # Select the known input at the current time step
            input = inputs_future[:, n-1, :]
            # Use the last prediction as unknown input and combine it with the known input
            # x.shape => (batch, features)
            x = tf.concat([prediction, input], axis=1)
            # Execute one lstm step
            x, state1 = self.rnn_cell1(x, states=state1,
                                     training=training)
            x, state2 = self.rnn_cell2(x, states=state2,
                                     training=training)
            # Convert the lstm output to a prediction.
            prediction = self.dense(x)
            # Add the prediction to the output.
            predictions.append(prediction)
        # predictions.shape => (time, batch, features)
        predictions = tf.stack(predictions)
        # predictions.shape => (batch, time, features)
        predictions = tf.transpose(predictions, [1, 0, 2])
        return predictions


In [ ]:
model_name = 'ARLSTM 96x32 + wx'

stacked_model = FeedBackStacked(units1=96, units2=32, out_steps=PREDICTION_STEPS)

history[model_name] = compile_and_fit(stacked_model, weather_window)
IPython.display.clear_output()

val_performance[model_name] = stacked_model.evaluate(weather_window.val, verbose=0)
test_performance[model_name] = stacked_model.evaluate(weather_window.test, verbose=1)

model_loss(history[model_name])

#### Model Performance Comparison

In [ ]:
x = np.arange(len(test_performance))
width = 0.3

mae_index = arlstm_model.metrics_names.index('mae')
r2_index = arlstm_model.metrics_names.index('r_square')

val_r2 = [v[r2_index] for v in val_performance.values()]
test_r2 = [v[r2_index] for v in test_performance.values()]

plt.bar(x - 0.17, val_r2, width, label='Validation')
plt.bar(x + 0.17, test_r2, width, label='Test')
plt.xticks(ticks=x, labels=test_performance.keys(),
           rotation=45, ha='right')
plt.ylabel('R² score (average over all outputs)')
_ = plt.legend()


In [ ]:
print(f'{"Model on test set":20s}    MAE     R²')
print('-------------------- ------ ------')
for name, value in test_performance.items():
    print(f'{name+":":20s} {value[mae_index]:0.4f} {value[r2_index]:0.4f}')


## Cross Validation

- Cross validate the model: Split the data set into $k$ subsets and train the model $k-1$ times, while taking one subset as validation set and the other subsets as training set. Reset the model after each training and compare the scores afterwards

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

NUM_FOLDS = 5

# Separate the dataframe into training and test set
n = len(df2)
df3_train = df2[0:int(n*0.9)]
df3_test = df2[int(n*0.9):]

scaler = RobustScaler()
# fit scaler to training data
scaler.fit(df3_train)
# scale all sets according to train set, preserve data frames
df3_train = pd.DataFrame(scaler.transform(df3_train),
                         columns=df2.columns, index=df3_train.index)
df3_test = pd.DataFrame(scaler.transform(df3_test),
                        columns=df2.columns, index=df3_test.index)

# Define the K-fold cross validator
tsplit = TimeSeriesSplit(n_splits=NUM_FOLDS)
cv_val_metrics = []
cv_test_metrics = []

# K-fold cross validation model evaluation
for fold_no, (train_index, val_index) in enumerate(tsplit.split(df3_train)):
    df3_cv_train = df3_train.iloc[train_index]
    df3_cv_val = df3_train.iloc[val_index]

    # Create Window for current fold
    cv_window = WindowGenerator(
        past_width=PAST_STEPS, future_width=PREDICTION_STEPS,
        label_columns=prediction_features,
        precise_columns=precise_features,
        forecast_columns=forecast_features,
        train_df=df3_cv_train, val_df=df3_cv_val, test_df=df3_test
    )

    # Create the model to be evaluated (with freshly initialized weights)
    cv_model = FeedBackGRU(units=32, out_steps=PREDICTION_STEPS)

    # Generate a print
    print(f'Training for fold {fold_no+1}/{NUM_FOLDS}...')

    # Compile and fit model
    compile_and_fit(cv_model, cv_window, epochs=25,
                    stop_early=True, patience=4, verbose=0)

    # Evaluate model
    print(f'Evaluate model for fold {fold_no+1} on validation set:')
    cv_val_metrics.append(cv_model.evaluate(cv_window.val, verbose=2))
    print(f'Evaluate model for fold {fold_no+1} on test set:')
    cv_test_metrics.append(cv_model.evaluate(cv_window.test, verbose=2))
    print('------------------------------------------------------------------------')


Visualize the chosen cross validation method:

In [ ]:
fig, ax = plt.subplots()
for fold_no, (train_index, val_index) in enumerate(tsplit.split(df3_train)):
    ax.scatter(
        train_index,
        [fold_no] * len(train_index),
        marker="|",
        c='C0')
    ax.scatter(
        val_index,
        [fold_no] * len(val_index),
        marker="|",
        c='C3')
# Formatting
_ = ax.set(yticks=np.arange(NUM_FOLDS),
       yticklabels=np.arange(NUM_FOLDS)+1,
       xlabel='Dataset index',
       ylabel='CV iteration',
       title='Cross Validation')


Evaluate the folds:

In [ ]:
r2_index = cv_model.metrics_names.index('r_square')
print('-------------------')
print(f'Fold Val R² Test R²')
print('---- ------ -------')
for i, test_metric in enumerate(cv_test_metrics):
    val_metric = cv_val_metrics[i]
    print(f'{i+1} {val_metric[r2_index]: 9.4f} {test_metric[r2_index]: 6.4f}')
print('---- ------ -------')
print(f'AVG {np.average(cv_val_metrics, axis=0)[r2_index]: 7.4f} {np.average(cv_test_metrics, axis=0)[r2_index]: 6.4f}')
print('-------------------')